<a href="https://colab.research.google.com/github/akshayj2518/Chat-with-files-RAG-application/blob/main/RAG_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
!pip install -q scipy
!pip install -q flash_attn
!pip install PyMuPDF
!pip install -q -U google-generativeai
!pip install chromadb==0.3.29

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.5/978.5 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 82.6 MB/s eta

In [2]:
from sentence_transformers import SentenceTransformer
from google.colab import files
import fitz
import google.generativeai as genai
import chromadb
from chromadb.config import Settings
query_prompt_name = "s2p_query"
model = SentenceTransformer('all-MiniLM-L6-v2')

chroma_client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory="./chroma_db"))
######################## model to create embeddings ########################
# other models: https://huggingface.co/spaces/mteb/leaderboard

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
uploaded = files.upload()

Saving Things to Study.docx to Things to Study.docx


In [4]:
file_name = list(uploaded.keys())[0]
doc = fitz.open(file_name)
txt = ""
for page in doc:
        txt += page.get_text() + "\n"
paragraphs = txt.split('\n\n') # chunking according to paragraphs
doc.close()

# Displaying all the chunks
import textwrap
for i, paragraph in enumerate(paragraphs):
  wrapped_text = textwrap.fill(paragraph, width=100)
  print("-" * 100)
  print(wrapped_text)
  print("-" * 100)


----------------------------------------------------------------------------------------------------
Things to Learn : Design : Review design of Facebook/Uber/whatsup etc. Data Structure Algorithm
Design Patterns (https://www.dofactory.com/net/design-patterns ) Creation Behavioural Structural
References : https://www.educative.io/ Refer Gaurav SEN videos
(https://get.interviewready.io/courses/ system-design-interview-prep) Refer Alex Ryan for system
design . Refer Solid principles(thought works) Major Design patterns used in industry. [Source] -
Pluralsight – Design Patterns in C# Technology Database : 2 hours per day – for 4 weeks. RDBMS = SQL
Server-Brush up. (8-12 hours) Postgres SQL – open source
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
No SQL (Cassandra – Table Format) [Source] - https:// academy.datastax.com/ - Contains i

In [5]:
embedded_docs = model.encode(paragraphs, normalize_embeddings=True) # working as an in memory Vector Database
# Create a collection in ChromaDB to store embeddings and their corresponding texts
collection_name = "pdf_documents"
collection = chroma_client.create_collection(name=collection_name)

shape = embedded_docs.shape
print("Number of chunks: " + str(shape[0]) + " Embedding Dimension: " + str(shape[1]))
embedded_docs[0]

Number of chunks: 8 Embedding Dimension: 384


array([-1.73045509e-02, -7.15204468e-03, -4.87527773e-02, -7.47947954e-03,
       -4.98362035e-02, -5.51983416e-02, -4.03183103e-02,  2.58934926e-02,
       -1.10833451e-01,  4.74870875e-02, -8.45424309e-02,  8.60131457e-02,
        2.19085328e-02, -3.42910998e-02,  1.46238990e-02,  2.18816660e-02,
        2.12677922e-02, -3.77800316e-02,  5.77556230e-02, -5.38681075e-02,
        2.54268181e-02, -6.79916441e-02,  2.66022533e-02,  2.83909142e-02,
       -2.43246276e-02,  9.04978290e-02,  5.47038019e-02, -2.59539261e-02,
        5.22035472e-02, -4.39630039e-02, -5.16112372e-02,  7.18105286e-02,
        3.03641576e-02,  3.58720198e-02, -7.73553625e-02,  2.26235539e-02,
        1.06510095e-01,  2.52991598e-02,  1.08234499e-05, -1.22891711e-02,
       -7.97892064e-02, -3.92954089e-02, -3.26836878e-03,  5.51856682e-02,
        6.17129244e-02, -3.14449482e-02, -5.78441285e-02, -6.17077015e-02,
       -1.33961122e-02,  3.25251110e-02, -1.02803662e-01, -5.30417897e-02,
        3.18906233e-02, -

In [6]:
# Add embeddings and their corresponding paragraphs to the collection with unique IDs
for i, embedding in enumerate(embedded_docs):
    collection.add(
        ids=[f"doc_{i}"],  # Unique ID for each document (e.g., "doc_0", "doc_1", ...)
        documents=[paragraphs[i]],
        embeddings=[embedding.tolist()],  # Convert numpy array to list for storage
        metadatas=[{"index": i}]  # Optional metadata, can include more info if needed
    )

In [7]:
# query = "Give me the link to learn SOLID principals?"
query = input("Please enter your query: ")
embedded_query = model.encode(query, normalize_embeddings=True)
embedded_query.shape


Please enter your query: Give me a list of top 10 topics to study


(384,)

In [8]:
# Perform similarity search in ChromaDB
results = collection.query(
    query_embeddings=[embedded_query.tolist()],
    n_results=3  # Retrieve top 3 results
)
# import numpy as np
# similarities = np.dot(embedded_docs, embedded_query.T)
# similarities.shape
# similarities

In [9]:
# Extracting relevant contexts from results
CONTEXT = ""
for idx in results['documents'][0]:
    wrapped_text = textwrap.fill(idx, width=100)
    print("-" * 100)
    print(wrapped_text)
    print("-" * 100)
    CONTEXT += wrapped_text + "\n\n"

----------------------------------------------------------------------------------------------------
Things to Learn : Design : Review design of Facebook/Uber/whatsup etc. Data Structure Algorithm
Design Patterns (https://www.dofactory.com/net/design-patterns ) Creation Behavioural Structural
References : https://www.educative.io/ Refer Gaurav SEN videos
(https://get.interviewready.io/courses/ system-design-interview-prep) Refer Alex Ryan for system
design . Refer Solid principles(thought works) Major Design patterns used in industry. [Source] -
Pluralsight – Design Patterns in C# Technology Database : 2 hours per day – for 4 weeks. RDBMS = SQL
Server-Brush up. (8-12 hours) Postgres SQL – open source
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Node JS(optional but advisable if time permits) Ruby on Rails(Python - optional) Java (o

In [10]:
# top_3_idx = np.argsort(similarities)[-3:][::-1].tolist()
# top_3_idx


In [11]:
# CONTEXT = ""
# for idx in top_3_idx:
#     wrapped_text = textwrap.fill(paragraphs[idx], width=100)
#     print("-" * 100)
#     print(wrapped_text)
#     print("-" * 100)
#     CONTEXT += wrapped_text + "\n\n"

In [12]:
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [14]:
genai.configure(api_key='YOUR_GEMINI_KEY')
gemini_model = genai.GenerativeModel("gemini-1.5-flash")
gemini_model.count_tokens(prompt)
response = gemini_model.generate_content(prompt)
print(response.text)

Based on the provided context, here's a possible list of 10 top study topics, prioritizing those mentioned most frequently and those appearing crucial for system design:

1. **System Design:**  Reviewing designs of Facebook/Uber/WhatsApp is explicitly mentioned as a key area.
2. **Data Structures and Algorithms:**  Fundamental to efficient system design.
3. **Design Patterns:** (Creational, Behavioral, Structural)  Essential for building robust and scalable systems.
4. **Databases (RDBMS & NoSQL):** SQL Server, PostgreSQL, and NoSQL databases like Redis are all highlighted. Understanding when to use each is vital.
5. **Microservices:**  The provided links emphasize the importance of understanding this architectural style.
6. **Caching (Redis):**  Specifically mentioned as a key technology for improving performance.
7. **Message Queues (RabbitMQ, Kafka):**  Essential for handling asynchronous communication and event-driven architectures.
8. **REST APIs:**  Fundamental for building web s